<a href="https://colab.research.google.com/github/nikimakarov/ARC25RELOAD/blob/main/nb/Qwen3_(4B)-GRPO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News


Unsloth's [Docker image](https://hub.docker.com/r/unsloth/unsloth) is here! Start training with no setup & environment issues. [Read our Guide](https://docs.unsloth.ai/new/how-to-train-llms-with-unsloth-and-docker).

[gpt-oss RL](https://docs.unsloth.ai/new/gpt-oss-reinforcement-learning) is now supported with the fastest inference & lowest VRAM. Try our [new notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/gpt-oss-(20B)-GRPO.ipynb) which creates kernels!

Introducing [Vision](https://docs.unsloth.ai/new/vision-reinforcement-learning-vlm-rl) and [Standby](https://docs.unsloth.ai/basics/memory-efficient-rl) for RL! Train Qwen, Gemma etc. VLMs with GSPO - even faster with less VRAM.

Unsloth now supports Text-to-Speech (TTS) models. Read our [guide here](https://docs.unsloth.ai/basics/text-to-speech-tts-fine-tuning).

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


In [1]:
import os

current_path = os.getcwd()
print("Current path:", current_path)

Current path: /content


In [2]:
import zipfile
import os

# Path to your ZIP file
zip_path = "ARC_RELOAD_V3.zip"
# Directory where you want to extract files
extract_dir = "ARC_RELOAD_V3"

# Make sure the output directory exists
os.makedirs(extract_dir, exist_ok=True)

# Unzip
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print(f"Extracted all files to '{extract_dir}'")

KeyboardInterrupt: 

### Installation

In [3]:
%%capture
import os
os.environ["UNSLOTH_VLLM_STANDBY"] = "1" # [NEW] Extra 30% context lengths!
if "COLAB_" not in "".join(os.environ.keys()):
    # If you're not in Colab, just use pip install or uv pip install
    !pip install unsloth vllm
else:
    pass # For Colab / Kaggle, we need extra instructions hidden below \/

In [ ]:
#@title Colab Extra Install { display-mode: "form" }
%%capture
import os
!pip install --upgrade -qqq uv
if "COLAB_" not in "".join(os.environ.keys()):
    # If you're not in Colab, just use pip install!
    !pip install unsloth vllm
else:
    try: import numpy; get_numpy = f"numpy=={numpy.__version__}"
    except: get_numpy = "numpy"
    try: import subprocess; is_t4 = "Tesla T4" in str(subprocess.check_output(["nvidia-smi"]))
    except: is_t4 = False
    get_vllm, get_triton = ("vllm==0.9.2", "triton==3.2.0") if is_t4 else ("vllm==0.10.2", "triton")
    !uv pip install -qqq --upgrade \
        unsloth {get_vllm} {get_numpy} torchvision bitsandbytes xformers
    !uv pip install -qqq {get_triton}
!uv pip install transformers==4.55.4
!uv pip install --no-deps trl==0.22.2

### Unsloth

Goal: To convert `Qwen3-4B-Base` into a reasoning model via GRPO by using OpenR1's Math dataset.

We first pre fine-tune the model to make GRPO skip trying to match formatting - this speeds GRPO up.

In [4]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Can increase for longer reasoning traces
lora_rank = 32 # Larger rank = smarter, but slower

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen3-4B-Base",
    max_seq_length = max_seq_length,
    load_in_4bit = False, # False for LoRA 16bit
    fast_inference = True, # Enable vLLM fast inference
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.8, # Reduce if out of memory
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha = lora_rank*2, # *2 speeds up training
    use_gradient_checkpointing = "unsloth", # Reduces memory usage
    random_state = 3407,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

INFO 10-14 21:28:10 [__init__.py:244] Automatically detected platform cuda.
ERROR 10-14 21:28:15 [fa_utils.py:57] Cannot use FA version 2 is not supported due to FA2 is only supported on devices with compute capability >= 8
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 10-14 21:28:35 [vllm_utils.py:689] Unsloth: Patching vLLM v1 graph capture
INFO 10-14 21:28:35 [vllm_utils.py:717] Unsloth: Patching vLLM v0 graph capture
==((====))==  Unsloth 2025.10.3: Fast Qwen3 patching. Transformers: 4.55.4. vLLM: 0.9.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/Qwen3-4B-Base with actual GPU utilization = 89.15%
Unslot

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


INFO 10-14 21:29:44 [default_loader.py:272] Loading weights took 37.89 seconds
INFO 10-14 21:29:44 [punica_selector.py:19] Using PunicaWrapperGPU.
INFO 10-14 21:29:45 [model_runner.py:1203] Model loading took 7.6764 GiB and 39.458799 seconds
INFO 10-14 21:29:52 [worker.py:294] Memory profiling takes 5.97 seconds
INFO 10-14 21:29:52 [worker.py:294] the current vLLM instance can use total_gpu_memory (14.74GiB) x gpu_memory_utilization (0.89) = 13.14GiB
INFO 10-14 21:29:52 [worker.py:294] model weights take 7.68GiB; non_torch_memory takes 0.03GiB; PyTorch activation peak memory takes 1.05GiB; the rest of the memory reserved for KV Cache is 4.39GiB.
INFO 10-14 21:29:52 [executor_base.py:113] # cuda blocks: 1996, # CPU blocks: 0
INFO 10-14 21:29:52 [executor_base.py:118] Maximum concurrency for 2048 tokens per request: 15.59x
INFO 10-14 21:29:53 [vllm_utils.py:722] Unsloth: Running patched vLLM v0 `capture_model`.
INFO 10-14 21:29:53 [model_runner.py:1513] Capturing cudagraphs for decoding.

Capturing CUDA graph shapes:   0%|          | 0/27 [00:00<?, ?it/s]

INFO 10-14 21:30:00 [model_runner.py:1671] Graph capturing finished in 8 secs, took 0.31 GiB
INFO 10-14 21:30:00 [vllm_utils.py:729] Unsloth: Patched vLLM v0 graph capture finished in 8 secs.
INFO 10-14 21:30:02 [llm_engine.py:428] init engine (profile, create kv cache, warmup model) took 16.53 seconds
Unsloth: Just some info: will skip parsing ['post_feedforward_layernorm', 'layer_norm2', 'pre_feedforward_layernorm', 'layer_norm1', 'norm1', 'input_layernorm', 'k_norm', 'post_layernorm', 'norm2', 'q_norm', 'attention_norm', 'ffn_norm', 'post_attention_layernorm']
Unsloth: Just some info: will skip parsing ['post_feedforward_layernorm', 'layer_norm2', 'pre_feedforward_layernorm', 'cross_attn_input_layernorm', 'layer_norm1', 'norm1', 'cross_attn_post_attention_layernorm', 'input_layernorm', 'k_norm', 'post_layernorm', 'norm2', 'q_norm', 'attention_norm', 'ffn_norm', 'post_attention_layernorm']


Unsloth 2025.10.3 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


### Pre fine-tuning for formatting
We now use a subset of NVIDIA's [Open Math Reasoning dataset](https://huggingface.co/datasets/nvidia/OpenMathReasoning) which was filtered to only include high quality DeepSeek R1 traces.

We'll only filter ~59 or so examples to first "prime" / pre fine-tune the model to understand our custom GRPO formatting.

In [11]:
import json
import pandas as pd
from datasets import Dataset
from trl import SFTTrainer

from accelerate.logging import get_logger
from accelerate import Accelerator

In [12]:
# Paths to datasets
DATASETS_DIR = '/content/ARC_RELOAD_V3/ARC_RELOAD_COPY_OCT_12/output/datasets_qwen25'

TASK1_TRAIN = f'{DATASETS_DIR}/task1_train.parquet'
TASK1_VAL = f'{DATASETS_DIR}/task1_validation.parquet'

TASK2_TRAIN = f'{DATASETS_DIR}/task2_train.parquet'
TASK2_VAL = f'{DATASETS_DIR}/task2_validation.parquet'

# Load config from datasets
with open(f'{DATASETS_DIR}/config.json', 'r') as f:
    dataset_config = json.load(f)

print("Dataset Configuration:")
print(f"  Model: {dataset_config['model_path']}")
print(f"  Grid Encoder: {dataset_config['grid_encoder']}")
print(f"  Max Seq Len: {dataset_config['max_seq_len']}")

print("Loading Task 1 datasets...")
task1_train_df = pd.read_parquet(TASK1_TRAIN)
task1_val_df = pd.read_parquet(TASK1_VAL)

print(f"  Task 1 Train: {len(task1_train_df)} samples")
print(f"  Task 1 Val: {len(task1_val_df)} samples")

# Convert to HuggingFace Datasets (only text column needed for training)
task1_train_dataset = Dataset.from_pandas(task1_train_df[['text']])
task1_val_dataset = Dataset.from_pandas(task1_val_df[['text', 'ground_truth_output']])

print("\nTask 1 datasets ready!")

Dataset Configuration:
  Model: Qwen/Qwen2.5-0.5B-Instruct
  Grid Encoder: GridShapeEncoder(RowNumberEncoder(MinimalGridEncoder()))
  Max Seq Len: 4096
Loading Task 1 datasets...
  Task 1 Train: 974 samples
  Task 1 Val: 124 samples

Task 1 datasets ready!


In [13]:
task1_train_dataset

Dataset({
    features: ['text'],
    num_rows: 974
})

In [14]:
task1_train_dataset

Dataset({
    features: ['text'],
    num_rows: 974
})

In [25]:
import re
train_dataset = pd.DataFrame(task1_train_dataset)  # or your ARC dataset
eval_dataset = pd.DataFrame(task1_val_dataset)

def format_arc(x):
    text = x["text"]
    matches = re.findall(r"<\|im_start\|\>(.*?)\n(.*?)<\|im_end\|\>", text, re.DOTALL)
    messages = [{"role": role.strip(), "content": content.strip()} for role, content in matches]
    return messages

train_dataset["Messages"] = train_dataset.apply(format_arc, axis=1)
eval_dataset["Messages"] = eval_dataset.apply(format_arc, axis=1)

In [26]:
train_dataset

,text,Messages
0,<|im_start|>system\nYou are a helpful assistan...,"[{'role': 'system', 'content': 'You are a help..."
1,<|im_start|>system\nYou are a helpful assistan...,"[{'role': 'system', 'content': 'You are a help..."
2,<|im_start|>system\nYou are a helpful assistan...,"[{'role': 'system', 'content': 'You are a help..."
3,<|im_start|>system\nYou are a helpful assistan...,"[{'role': 'system', 'content': 'You are a help..."
4,<|im_start|>system\nYou are a helpful assistan...,"[{'role': 'system', 'content': 'You are a help..."
...,...,...
969,<|im_start|>system\nYou are a helpful assistan...,"[{'role': 'system', 'content': 'You are a help..."
970,<|im_start|>system\nYou are a helpful assistan...,"[{'role': 'system', 'content': 'You are a help..."
971,<|im_start|>system\nYou are a helpful assistan...,"[{'role': 'system', 'content': 'You are a help..."
972,<|im_start|>system\nYou are a helpful assistan...,"[{'role': 'system', 'content': 'You are a help..."


In [27]:
eval_dataset

,text,ground_truth_output,Messages
0,<|im_start|>system\nYou are a helpful assistan...,"[[8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,...","[{'role': 'system', 'content': 'You are a help..."
1,<|im_start|>system\nYou are a helpful assistan...,"[[0, 0, 5, 0, 0, 0, 0], [0, 0, 1, 1, 1, 0, 0],...","[{'role': 'system', 'content': 'You are a help..."
2,<|im_start|>system\nYou are a helpful assistan...,"[[0, 1, 0, 2, 0, 7, 0, 5, 0, 0, 2, 0, 8, 0, 6,...","[{'role': 'system', 'content': 'You are a help..."
3,<|im_start|>system\nYou are a helpful assistan...,"[[0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[{'role': 'system', 'content': 'You are a help..."
4,<|im_start|>system\nYou are a helpful assistan...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[{'role': 'system', 'content': 'You are a help..."
...,...,...,...
119,<|im_start|>system\nYou are a helpful assistan...,"[[2, 3, 3, 3, 2, 3, 3, 3], [3, 2, 3, 2, 3, 2, ...","[{'role': 'system', 'content': 'You are a help..."
120,<|im_start|>system\nYou are a helpful assistan...,"[[7, 7, 7, 3], [7, 7, 3, 7], [7, 6, 7, 8], [6,...","[{'role': 'system', 'content': 'You are a help..."
121,<|im_start|>system\nYou are a helpful assistan...,"[[0, 0, 0, 6, 0, 0, 0, 3, 0, 0, 0, 7, 0, 0, 0,...","[{'role': 'system', 'content': 'You are a help..."
122,<|im_start|>system\nYou are a helpful assistan...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, ...","[{'role': 'system', 'content': 'You are a help..."


In [28]:
chat_template = (
    "{% for message in messages %}"
        "<|im_start|>{{ message['role'] }}\n"
        "{{ message['content'] }}<|im_end|>\n"
    "{% endfor %}"
    "{% if add_generation_prompt %}<|im_start|>assistant\n{% endif %}"
)

tokenizer.chat_template = chat_template

Check to see if it worked:

In [29]:
train_dataset["N"] = train_dataset["Messages"].apply(lambda x: len(tokenizer.apply_chat_template(x)))
train_dataset = train_dataset.loc[train_dataset["N"] <= max_seq_length/4].copy()
train_dataset.shape

(46, 3)

In [30]:
#eval_dataset["N"] = eval_dataset["Messages"].apply(lambda x: len(tokenizer.apply_chat_template(x)))
#eval_dataset.shape

We then tokenize the messages and convert it to a Hugging Face compatible dataset format:

In [31]:
from datasets import Dataset

train_dataset["text"] = tokenizer.apply_chat_template(train_dataset["Messages"].values.tolist(), tokenize = False)
train_dataset = Dataset.from_pandas(train_dataset)
train_dataset

Dataset({
    features: ['text', 'Messages', 'N', '__index_level_0__'],
    num_rows: 46
})

In [58]:
train_dataset

NameError: name 'train_dataset' is not defined

In [32]:
#eval_dataset["text"] = tokenizer.apply_chat_template(eval_dataset["Messages"].values.tolist(), tokenize = False)
#eval_dataset = Dataset.from_pandas(eval_dataset)
#eval_dataset

Let's now pre fine-tune the model so it follows our custom GRPO formatting!

In [33]:
from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    args = SFTConfig(
        dataset_text_field = "text",
        save_steps=100,
        gradient_accumulation_steps = 1, # Use GA to mimic batch size!
        warmup_steps = 5,
        warmup_ratio=0.05,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=4,
        num_train_epochs = 2, # Set this for 1 full training run.
        learning_rate = 2e-4, # Reduce to 2e-5 for long training runs
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/46 [00:00<?, ? examples/s]

In [34]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 46 | Num Epochs = 2 | Total steps = 12
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 1 x 1) = 8
 "-____-"     Trainable parameters = 66,060,288 of 4,088,528,384 (1.62% trained)


Step,Training Loss
1,1.213400
2,1.248100
3,1.181600
4,0.951200
5,0.909300
6,0.767600
7,0.591000
8,0.561300
9,0.368600
10,0.339900


Unsloth: Will smartly offload gradients to save VRAM!


TrainOutput(global_step=12, training_loss=0.7355785891413689, metrics={'train_runtime': 93.9776, 'train_samples_per_second': 0.979, 'train_steps_per_second': 0.128, 'total_flos': 1032624599162880.0, 'train_loss': 0.7355785891413689, 'epoch': 2.0})

Let's check if the model has learnt to follow the custom format:

In [35]:
text = tokenizer.apply_chat_template(
    train_dataset[0]["Messages"][:2],
    tokenize = False,
    add_generation_prompt = True, # Must add for generation
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    temperature = 0,
    max_new_tokens = 1024,
    streamer = TextStreamer(tokenizer, skip_prompt = False),
)

<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
Let's see if you can solve this simple Abstraction and Reasoning Challenge (ARC) task.
Below there are some input-output grid examples that define the task.
Your job is to understand the transformation between the input and the output and apply it to the test input grid.
The transformations are always based on the following priors: objectness, goal-directed, numbers & counting, and basic geometry & topology.

## Example 1

### Input

```grid shape: 2x2
1 79
2 43
```

### Output

```grid shape: 6x6
1 797979
2 434343
3 979797
4 343434
5 797979
6 434343
```

## Example 2

### Input

```grid shape: 2x2
1 86
2 64
```

### Output

```grid shape: 6x6
1 868686
2 646464
3 686868
4 464646
5 868686
6 646464
```

## Test case

### Input

```grid shape: 2x2
1 32
2 78
```<|im_end|>
<|im_start|>assistant
### Output

```grid shape: 6x6
1 323232
2 787878
3 232323
4 878787
5 323232
6 787878
```.EditorButton
Below there are some i

KeyboardInterrupt: 

In [ ]:
del train_dataset

Yes it did follow the formatting! Great! Let's remove some items before the GRPO step

In [53]:
torch.cuda.empty_cache()
import gc
gc.collect()

1281

### Data Prep
<a name="Data"></a>

We're using Hugging Face's [Open R1 Math dataset](https://huggingface.co/datasets/open-r1/DAPO-Math-17k-Processed). You can also utilize OpenAI's famous [GSM8K dataset](https://huggingface.co/datasets/openai/gsm8k)

In [223]:
task1_train_dataset

Dataset({
    features: ['text'],
    num_rows: 974
})

In [224]:
task1_train_dataset[0]

{'text': "<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n<|im_start|>user\nLet's see if you can solve this simple Abstraction and Reasoning Challenge (ARC) task.\nBelow there are some input-output grid examples that define the task.\nYour job is to understand the transformation between the input and the output and apply it to the test input grid.\nThe transformations are always based on the following priors: objectness, goal-directed, numbers & counting, and basic geometry & topology.\n\n## Example 1\n\n### Input\n\n```grid shape: 2x2\n1 79\n2 43\n```\n\n### Output\n\n```grid shape: 6x6\n1 797979\n2 434343\n3 979797\n4 343434\n5 797979\n6 434343\n```\n\n## Example 2\n\n### Input\n\n```grid shape: 2x2\n1 86\n2 64\n```\n\n### Output\n\n```grid shape: 6x6\n1 868686\n2 646464\n3 686868\n4 464646\n5 868686\n6 646464\n```\n\n## Test case\n\n### Input\n\n```grid shape: 2x2\n1 32\n2 78\n```<|im_end|>\n<|im_start|>assistant\n### Output\n\n```grid shape: 6x6\n1 323232\n2 787878\n3 2

In [225]:
task1_val_dataset

Dataset({
    features: ['text', 'ground_truth_output'],
    num_rows: 124
})

In [226]:
task1_val_dataset[0]

{'text': "<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n<|im_start|>user\nLet's see if you can solve this simple Abstraction and Reasoning Challenge (ARC) task.\nBelow there are some input-output grid examples that define the task.\nYour job is to understand the transformation between the input and the output and apply it to the test input grid.\nThe transformations are always based on the following priors: objectness, goal-directed, numbers & counting, and basic geometry & topology.\n\n## Example 1\n\n### Input\n\n```grid shape: 5x13\n1 3333333333333\n2 3222222222223\n3 3213131333123\n4 3222222222223\n5 3333333333333\n```\n\n### Output\n\n```grid shape: 5x13\n1 3333333333333\n2 3222222222223\n3 3213131313123\n4 3222222222223\n5 3333333333333\n```\n\n## Example 2\n\n### Input\n\n```grid shape: 21x22\n1 4444444444444444444444\n2 4222222222222222222224\n3 4233433433433433443324\n4 4233433433433433443324\n5 4222222222222222222224\n6 4444444444444444444444\n7 4222222222222222

In [259]:
import re

ARC_GRID_PATTERN = re.compile(
    r"grid shape:\s*\d+x\d+[\s\S]*?(?:\d\s[\d\s]+)+",
    flags=re.MULTILINE | re.IGNORECASE,
)

def _extract_response(completion):
    """Helper to handle both string and list[dict] completions."""
    if isinstance(completion, str):
        return completion
    elif isinstance(completion, list) and len(completion) > 0:
        if isinstance(completion[0], dict) and "content" in completion[0]:
            return completion[0]["content"]
        elif isinstance(completion[0], str):
            return completion[0]
    return ""

# ============================================================
# 1️⃣ FORMAT CHECKS
# ============================================================

def check_arc_format_exactly(completions, **kwargs):
    scores = []
    for completion in completions:
        response = _extract_response(completion)
        scores.append(3.0 if ARC_GRID_PATTERN.search(response) else 0.0)
    return scores


def check_arc_format_approximately(completions, **kwargs):
    scores = []
    for completion in completions:
        response = _extract_response(completion)
        score = 0.0
        if "grid shape" in response: score += 0.5
        if "```" in response: score += 0.5
        if any(c.isdigit() for c in response): score += 0.5
        scores.append(score)
    return scores

# ============================================================
# 2️⃣ ANSWER MATCHING
# ============================================================

def check_arc_answer(prompts, completions, answers, **kwargs):
    scores = []
    for completion, true_answer in zip(completions, answers):
        response = _extract_response(completion)

        gen_nums  = re.findall(r"\d+", response)
        true_nums = re.findall(r"\d+", true_answer)

        if not gen_nums or not true_nums:
            scores.append(-2.0)
            continue

        overlap = sum(1 for g, t in zip(gen_nums, true_nums) if g == t)
        ratio = overlap / max(len(true_nums), 1)

        if ratio > 0.95: score = 5.0
        elif ratio > 0.8: score = 3.0
        elif ratio > 0.5: score = 1.0
        else: score = -1.0
        scores.append(score)
    return scores

# ============================================================
# 3️⃣ MAIN COMBINED FUNCTION
# ============================================================

def check_arc_overall(prompts, completions, **kwargs):
    answers = kwargs.get("answer", {})
    print(answers)

    # Выровнять по минимальной длине
    min_len = min(len(completions), len(answers))
    completions = completions[:min_len]
    answers = answers[:min_len]

    exact  = check_arc_format_exactly(completions)
    approx = check_arc_format_approximately(completions)
    grid   = check_arc_answer(prompts, completions, answers)
    print("DATA:")
    print(prompts)
    print(completions)
    print(answers)

    scores = [e + a + g for e, a, g in zip(exact, approx, grid)]
    return scores

In [260]:
# Prepare your datasets
train_filtered, eval_filtered, max_train_len, max_eval_len = prepare_arc_dataset(
    task1_train_dataset,
    task1_val_dataset,
    tokenizer
)

➡️ Converting to Hugging Face Datasets...


TypeError: expected string or bytes-like object, got 'Series'

In [261]:
import re

def format_arc_text(text):
    matches = re.findall(r"<\|im_start\|\>(.*?)\n(.*?)<\|im_end\|\>", text, re.DOTALL)
    messages = [{"role": role.strip(), "content": content.strip()} for role, content in matches]
    return messages

train_filtered = train_filtered.map(
    lambda x: {
        "tokens": tokenizer.apply_chat_template(
            format_arc_text(x["text"]),   # Форматируем из текстовой колонки
            add_generation_prompt=True,
            tokenize=True
        )
    },
    batched=False
)

Map:   0%|          | 0/876 [00:00<?, ? examples/s]

In [262]:
eval_filtered = eval_filtered.map(
    lambda x: {
        "tokens": tokenizer.apply_chat_template(
            format_arc_text(x["text"]),   # Форматируем из текстовой колонки
            add_generation_prompt=True,
            tokenize=True
        )
    },
    batched=False
)

Map:   0%|          | 0/111 [00:00<?, ? examples/s]

In [263]:
import numpy as np
maximum_length = int(np.quantile(train_filtered["L"], 0.7))
print("Max Length = ", maximum_length)


Max Length =  1739


In [264]:

maximum_length_eval = int(np.quantile(eval_filtered["L"], 0.2))
print("Max Length = ", maximum_length_eval)

Max Length =  1703


In [265]:
train_filtered_s = train_filtered.select(np.where(np.array(train_filtered["L"]) <= maximum_length)[0])

In [266]:
train_filtered_s.shape

(614, 4)

In [267]:
eval_filtered_s = eval_filtered.select(np.where(np.array(eval_filtered["L"]) <= maximum_length_eval)[0])

In [268]:
eval_filtered_s.shape

(23, 5)

In [269]:
def extract_ground_truth(example):
    try:
        # Берем контент третьего сообщения (assistant)
        answer = example["Messages"][2]["content"]
    except Exception:
        answer = ""
    return {"answer": answer}

eval_filtered_s = eval_filtered_s.map(extract_ground_truth)
train_filtered_s = train_filtered_s.map(extract_ground_truth)


Map:   0%|          | 0/23 [00:00<?, ? examples/s]

Map:   0%|          | 0/614 [00:00<?, ? examples/s]

In [270]:
eval_filtered_s[5]["answer"]

'### Output\n\n```grid shape: 12x13\n1 0000000000000\n2 0000000000000\n3 0007022222000\n4 0007000000244\n5 0007022222000\n6 0007000000000\n7 0009000000000\n8 0099000008000\n9 0900900000800\n10 0900900200082\n11 0900900000800\n12 0900900008000\n```'

In [271]:
def extract_prompt(example):
    # Извлекаем только system + user (до assistant)
    msgs = example["Messages"]
    prompt_msgs = [m for m in msgs if m["role"] != "assistant"]
    return tokenizer.apply_chat_template(prompt_msgs, add_generation_prompt=True, tokenize=False)

train_filtered_s = train_filtered_s.map(lambda x: {"prompt": extract_prompt(x)})
eval_filtered_s  = eval_filtered_s.map(lambda x: {"prompt": extract_prompt(x)})

Map:   0%|          | 0/614 [00:00<?, ? examples/s]

Map:   0%|          | 0/23 [00:00<?, ? examples/s]

In [272]:
eval_filtered_s[0]["prompt"]

"<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n<|im_start|>user\nLet's see if you can solve this simple Abstraction and Reasoning Challenge (ARC) task.\nBelow there are some input-output grid examples that define the task.\nYour job is to understand the transformation between the input and the output and apply it to the test input grid.\nThe transformations are always based on the following priors: objectness, goal-directed, numbers & counting, and basic geometry & topology.\n\n## Example 1\n\n### Input\n\n```grid shape: 12x9\n1 000000000\n2 100100000\n3 000000000\n4 000000000\n5 110000000\n6 000000000\n7 000000000\n8 000000000\n9 909000600\n10 000000000\n11 000000000\n12 880000030\n```\n\n### Output\n\n```grid shape: 12x9\n1 000000000\n2 100100100\n3 000000000\n4 000000000\n5 111111111\n6 000000000\n7 000000000\n8 000000000\n9 606060600\n10 000000000\n11 000000000\n12 333333330\n```\n\n## Example 2\n\n### Input\n\n```grid shape: 9x20\n1 00000000000000000000\n2 0000000000

In [273]:
eval_filtered_s.shape

(23, 7)

In [274]:
sample_idx = 0
prompts = [eval_filtered_s[sample_idx]["Messages"][:-1]]  # system + user
completions = [[{"content": "### Output\n```grid shape: 6x6\n1 023232\n2 787878\n```"}]]
inputs = {"answer": [eval_filtered_s[sample_idx]["answer"]]}

score = check_arc_overall(prompts, completions, inputs=inputs)
print("Reward score:", score)

INPUTS:
{'answer': ['### Output\n\n```grid shape: 9x21\n1 000000000000000000000\n2 002002002702002002002\n3 000000999999999999999\n4 000666666666666666666\n5 000000000000000000000\n6 600006000060010600006\n7 000000000000000000000\n8 008008008008008008008\n9 707070747070707070707\n```']}
ANSWERS:
['### Output\n\n```grid shape: 9x21\n1 000000000000000000000\n2 002002002702002002002\n3 000000999999999999999\n4 000666666666666666666\n5 000000000000000000000\n6 600006000060010600006\n7 000000000000000000000\n8 008008008008008008008\n9 707070747070707070707\n```']
DATA:
[[{'content': 'You are a helpful assistant.', 'role': 'system'}, {'content': "Let's see if you can solve this simple Abstraction and Reasoning Challenge (ARC) task.\nBelow there are some input-output grid examples that define the task.\nYour job is to understand the transformation between the input and the output and apply it to the test input grid.\nThe transformations are always based on the following priors: objectness, go

<a name="Train"></a>
### Train the model

Now set up GRPO Trainer and all configurations!

In [275]:

max_prompt_length = maximum_length + 1
max_completion_length = max_seq_length - max_prompt_length

from vllm import SamplingParams
vllm_sampling_params = SamplingParams(
    min_p = 0.1,
    top_p = 1.0,
    top_k = -1,
    seed = 3407,
    stop = [tokenizer.eos_token],
    include_stop_str_in_output = True,
)

from trl import GRPOConfig, GRPOTrainer
training_args = GRPOConfig(
    vllm_sampling_params = vllm_sampling_params,
    temperature = 1.0,
    learning_rate = 5e-6,
    weight_decay = 0.01,
    warmup_ratio = 0.1,
    lr_scheduler_type = "linear",
    optim = "adamw_8bit",
    logging_steps = 1,
    per_device_train_batch_size = 4,
    gradient_accumulation_steps = 1, # Increase to 4 for smoother training
    num_generations = 4, # Decrease if out of memory
    max_prompt_length = max_prompt_length,
    max_completion_length = max_completion_length,
    # num_train_epochs = 1, # Set to 1 for a full training run
    max_steps = 100,
    save_steps = 100,
    report_to = "none", # Can use Weights & Biases
    output_dir = "outputs",

    # For optional training + evaluation
    # fp16_full_eval = True,
    # per_device_eval_batch_size = 4,
    # eval_accumulation_steps = 1,
    # eval_strategy = "steps",
    # eval_steps = 1,
)

And let's run the trainer! If you scroll up, you'll see a table of rewards. The goal is to see the `reward` column increase!

You might have to wait 150 to 200 steps for any action. You'll probably get 0 reward for the first 100 steps. Please be patient!

| Step | Training Loss | reward    | reward_std | completion_length | kl       |
|------|---------------|-----------|------------|-------------------|----------|
| 1    | 0.000000      | 0.125000  | 0.000000   | 200.000000        | 0.000000 |
| 2    | 0.000000      | 0.072375  | 0.248112   | 200.000000        | 0.000000 |
| 3    | 0.000000      | -0.079000 | 0.163776   | 182.500000        | 0.000005 |


In [276]:
print(eval_filtered_s.column_names)
print(train_filtered_s.column_names)

['text', 'ground_truth_output', 'Messages', 'tokens', 'L', 'answer', 'prompt']
['text', 'Messages', 'tokens', 'L', 'answer', 'prompt']


In [277]:
# For optional training + evaluation
# new_dataset = dataset.train_test_split(test_size = 0.01)

trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        check_arc_overall
    ],
    reward_kwargs_field="answer",
    args = training_args,
    train_dataset = train_filtered_s,
    eval_dataset = eval_filtered_s

    # For optional training + evaluation
    # train_dataset = new_dataset["train"],
    # eval_dataset = new_dataset["test"],
)
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 614 | Num Epochs = 1 | Total steps = 100
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 1 x 1) = 4
 "-____-"     Trainable parameters = 66,060,288 of 4,088,528,384 (1.62% trained)


INPUTS:
{}


KeyError: 'answer'

<a name="Inference"></a>
### Inference
Now let's try the model we just trained! First, let's first try the model without any GRPO trained:

In [ ]:
text = "What is the sqrt of 101?"

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 1.0,
    top_k = 50,
    max_tokens = 1024,
)
output = model.fast_generate(
    [text],
    sampling_params = sampling_params,
    lora_request = None,
)[0].outputs[0].text

output

And now with the LoRA we just trained with GRPO - we first save the LoRA first!

In [ ]:
model.save_lora("grpo_saved_lora")

Verify LoRA is actually trained!

In [ ]:
from safetensors import safe_open

tensors = {}
with safe_open("grpo_saved_lora/adapter_model.safetensors", framework = "pt") as f:
    # Verify both A and B are non zero
    for key in f.keys():
        tensor = f.get_tensor(key)
        n_zeros = (tensor == 0).sum() / tensor.numel()
        assert(n_zeros.item() != tensor.numel())

Now we load the LoRA and test:

In [ ]:
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user",   "content": "What is the sqrt of 101?"},
]

text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
    tokenize = False,
)
from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 1.0,
    top_k = 50,
    max_tokens = 2048,
)
output = model.fast_generate(
    text,
    sampling_params = sampling_params,
    lora_request = model.load_lora("grpo_saved_lora"),
)[0].outputs[0].text

output

Our reasoning model is much better - it's not always correct, since we only trained it for an hour or so - it'll be better if we extend the sequence length and train for longer!

<a name="Save"></a>
### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False:
    model.save_pretrained("model")
    tokenizer.save_pretrained("model")
if False:
    model.push_to_hub("hf/model", token = "")
    tokenizer.push_to_hub("hf/model", token = "")


### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)

In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "",
    )

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in llama.cpp.

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Train your own reasoning model - Llama GRPO notebook [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-GRPO.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>
